In [1]:
import numpy as np
from astropy import units as u 
from astropy.io import fits
import matplotlib.pyplot as plt
import matplotlib as mpl
from astropy.table import QTable
import numpy as np

plt.style.use('paper.mplstyle')

In [2]:
root_dir = '/Users/abarnes/Dropbox/work/Smallprojects/galaxies'
table = QTable.read('%s/data_hstha_nebulae_catalogue/all/nebulae_catalogue.fits' %root_dir)
table_muse = QTable.read('%s/data_misc/Nebulae_catalogue_v4/Nebulae_catalogue_v4.fits' %root_dir)

In [3]:
table = table[table['HII_class_v3_MUSE']==1]
table_muse = table_muse[table_muse['HII_class_v3']==1]

In [4]:
# Calculate the percentiles for each "gal_name"
percentiles = [10, 50, 90]
columns = ['gal_name'] + [f'percentile_{p}' for p in percentiles]
data = []

for gal_name in np.unique(table['gal_name']):

    table_masked = table[table['gal_name'] == gal_name]
    table_muse_masked = table_muse[table_muse['gal_name'] == gal_name]

    gal_data = [gal_name.replace('NGC', 'NGC~').replace('IC', 'IC~')]
        
    p_lum = np.percentile(table_masked['HA6562_LUMINOSITY_HST'], percentiles).to('dex(erg/s)')
    p_rad = np.percentile(table_masked['radius_circ_pc'], percentiles)
    p_muse_rad = np.percentile(table_masked['region_circ_rad_pc_MUSE'], percentiles)
    p_muse_rad_ratio = np.percentile(table_masked['radius_circ_pc']/table_masked['region_circ_rad_pc_MUSE'], percentiles)

    for values in [p_lum, p_rad, p_muse_rad, p_muse_rad_ratio]:
        for value in values:
            gal_data.extend([value])
    
    data.append(gal_data)

In [5]:
# Calculate the percentiles for each "gal_name"
percentiles = [10, 50, 90]

gal_data = ['All']

p_lum = np.percentile(table['HA6562_LUMINOSITY_HST'], percentiles).to('dex(erg/s)')
p_rad = np.percentile(table['radius_circ_pc'], percentiles)
p_muse_rad = np.percentile(table['region_circ_rad_pc_MUSE'], percentiles)
p_muse_rad_ratio = np.percentile(table['radius_circ_pc']/table['region_circ_rad_pc_MUSE'], percentiles)

for values in [p_lum, p_rad, p_muse_rad, p_muse_rad_ratio]:
    for value in values:
        gal_data.extend([value])

data.append(gal_data)

In [6]:
# Create a new QTable with the calculated percentiles
names1 = ['gal_name']
names2 = [f'p{p}_lum' for p in percentiles]
names3 = [f'p{p}_rad' for p in percentiles]
names4 = [f'p{p}_muse_rad' for p in percentiles]
names5 = [f'p{p}_rad_ratio' for p in percentiles]

names = names1 + names2 + names3 + names4 + names5
new_table = QTable(rows=data, names=names)

new_table

gal_name,p10_lum,p50_lum,p90_lum,p10_rad,p50_rad,p90_rad,p10_muse_rad,p50_muse_rad,p90_muse_rad,p10_rad_ratio,p50_rad_ratio,p90_rad_ratio
,dex(erg / s),dex(erg / s),dex(erg / s),pc,pc,pc,pc,pc,pc,,,
str8,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
IC~5332,36.34992159520272,36.90373231082586,37.653467105814975,8.528219049502558,14.350416557230577,32.82415815280308,29.118096139435547,45.974175199373455,71.47809475813379,0.23611500201247465,0.3408435058929893,0.49617795423117206
NGC~0628,36.488172817057844,37.25203505080238,38.19496740708628,9.296423396959716,20.53979258560183,52.066960345825656,31.38803165149418,45.67629525153207,79.11366407717058,0.2685579850820873,0.4605241449534478,0.7193667347256512
NGC~1087,37.043302145495645,37.76009096563498,38.77998206132096,14.371175425367591,30.34034869591672,92.53198350499477,51.29710043866201,65.4630480859476,128.90059796653202,0.2618409828714189,0.48016268987708494,0.7685768892015531
NGC~1300,36.83019946989189,37.535882863823005,38.49329801461271,16.591881280264,32.27799178933874,82.75494667850211,58.76645141744895,62.33123445532192,98.55433512439103,0.26439115021592396,0.5078253439914133,0.8961863900737482
NGC~1365,37.33800396073866,38.14945472336546,39.56416859121111,17.488770109263623,42.36344409392414,138.97472798400332,77.93961781900917,98.12055139921425,174.9347012880643,0.2086258159742189,0.4542766332123595,0.8813480085767563
NGC~1385,37.28384731936523,38.14646961057564,39.20469419989987,17.899505667988887,44.200639636312594,121.67256848171291,47.10127380454329,76.53051145907668,145.3286122653777,0.3364561206960649,0.5870785896579597,0.9274561845822153
NGC~1433,37.1225304726672,37.77754718615889,38.56587101152621,8.657607772620512,24.142610511885557,55.09716268516814,58.54628242932918,76.59991791380654,124.52935728046278,0.13293933739560318,0.3213366704415592,0.4530573990543479
NGC~1512,36.93734141840625,37.47850340304309,38.48372929025732,15.540976680121545,25.075683846687816,74.93863446034528,79.12356420177082,83.68580617875041,126.91514887158725,0.17439825983662768,0.2956454333703235,0.64508127725292


In [7]:
names = ['gal_name',
        'p10_lum',
        'p50_lum',
        'p90_lum',
        'p10_rad',
        'p50_rad',
        'p90_rad',
        'p10_muse_rad',
        'p50_muse_rad',
        'p90_muse_rad',
        'p10_rad_ratio',
        'p50_rad_ratio',
        'p90_rad_ratio']

formats={
        'gal_name': '%s',
        'p10_lum': '%0.1f',
        'p50_lum': '%0.1f',
        'p90_lum': '%0.1f',
        'p10_rad': '%0.1f',
        'p50_rad': '%0.1f',
        'p90_rad': '%0.1f', 
        'p10_muse_rad': '%0.1f',
        'p50_muse_rad': '%0.1f',
        'p90_muse_rad': '%0.1f',
        'p10_rad_ratio': '%0.2f',
        'p50_rad_ratio': '%0.2f',
        'p90_rad_ratio': '%0.2f'
        }

new_table.write('./tabs/lumrad_props.tex', formats=formats, names=names, overwrite=True)
# new_table.write('./tabs/lumrad_props.fits', overwrite=True)
# new_table.write('./tabs/lumrad_props.dat', format='ascii', overwrite=True)